In [14]:
import json

#import distance data
with open(r'json/distance.json') as f:
    distance = json.load(f)

#import input data
with open(r'json/input.json') as f:
    input = json.load(f)


In [15]:
def distance_to(start:str, end:str, data):
    dis = data[start][end] 
    if (data[start][end] == "n/a"):
        return 0
    else:
        return dis